python 計算機科学新教本

In [9]:
def fib(n: int)->int:
    if n < 2:
        return n
    return fib(n-1) + fib(n-2)

from typing import Dict
memo: Dict[int, int] = {0:0, 1:1}
def fib3(n: int)->int:
    if n not in memo:
        memo[n] = fib3(n-1) + fib3(n-2)
    return memo[n]

from functools import lru_cache

@lru_cache(maxsize=None)
def fib4(n: int)->int:
    if n < 2:
        return n
    return fib4(n-1) + fib4(n-2)

def fib5(n: int)->int:
    if n == 0: return 0
    last: int = 0
    next: int = 1
    for _ in range(1, n):
        last, next = next, last + next
    return next

In [13]:
%%time
fib(40)

CPU times: user 35.2 s, sys: 0 ns, total: 35.2 s
Wall time: 35.2 s


102334155

In [14]:
%%time
fib3(40)

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 23.4 µs


102334155

In [15]:
%%time
fib4(40)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 23.4 µs


102334155

In [16]:
%%time
fib5(40)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 20.5 µs


102334155

In [99]:
class CompressedGene:
    def __init__(self, gene: str)->None:
        self._compress(gene)
    
    def _compress(self, gene: str)->None:
        self.bit_string: int = 1
        for nucleotide in gene.upper():
            # 2シフトして2ビットor演算＝末尾に追加
            self.bit_string <<= 2
            if nucleotide == "A":
                self.bit_string |= 0b00
            elif nucleotide == "C":
                self.bit_string |= 0b01
            elif nucleotide == "G":
                self.bit_string |= 0b10
            elif nucleotide == "T":
                self.bit_string |= 0b11
            else:
                raise ValueError("Invalid Nucleotide: {}".format(nucleotide))
    
    def decompress(self)->str:
        gene: str = ""
        for i in range(0, self.bit_string.bit_length() - 1, 2):
            bits: int = self.bit_string >> i & 0b11
            if bits == 0b00:
                gene += "A"
            elif bits == 0b01:
                gene += "C"
            elif bits == 0b10:
                gene += "G"
            elif bits == 0b11:
                gene += "T"
            else:
                raise ValueError("Invalid bits: {}".format(bits))
        return gene[::-1]

    def __str__(self)->str:
        return self.decompress()

In [100]:
from sys import getsizeof
original: str = "AAAAAGTAGATCCCCCCCCAGTACGGGGGG" * 100
print("original: {}bits".format(getsizeof(original)))
compressed: CompressedGene = CompressedGene(original)
print("compressed is: {}bits".format(getsizeof(compressed.bit_string)))
print("compress rate {}%".format(getsizeof(compressed.bit_string) / getsizeof(original)))

original: 3049bits
compressed is: 828bits
compress rate 0.2715644473597901%


In [114]:
from secrets import token_bytes
from typing import Tuple

def random_key(length: int)->int:
    tb:bytes = token_bytes(length)
    return int.from_bytes(tb, "big")

def encrypt(original: str)->Tuple[int, int]:
    original_bytes: bytes = original.encode()
    dummy: int = random_key(len(original_bytes))
    original_key: int = int.from_bytes(original_bytes, "big")
    encrypted: int = original_key ^ dummy
    return dummy, encrypted

def decrypt(key1: int, key2: int)->str:
    decrypted: int = key1 ^ key2
    temp: bytes = decrypted.to_bytes((decrypted.bit_length() + 7) // 8, "big")
    return temp.decode()

In [116]:
k, e = encrypt("ne mu su gi")
print(k, e)
result: str = decrypt(k, e)
print(result)

35689086810757900961294998 140085850249937596322663935
ne mu su gi


In [117]:
def calculate_pi(n_terms: int)->float:
    numerator: float = 4.0
    denominator: float = 1.0
    operation: float = 1.0
    pi: float = 0.0
    for _ in range(n_terms):
        pi += operation * (numerator / denominator)
        denominator += 2.0
        operation *= -1.0
    return pi

In [121]:
calculate_pi(10000000)

3.1415925535897915

In [131]:
from typing import TypeVar, Generic, List
T = TypeVar("T")

class Stack(Generic[T]):
    def __init__(self)->None:
        self._container: List[T] = []
            
    def push(self, item: T)->None:
        self._container.append(item)
        
    def pop(self)->T:
        return self._container.pop()
    
    def __repr__(self)->str:
        return repr(self._container)

In [141]:
disc_num: int = 3
tower_a: Stack[int] = Stack()
tower_b: Stack[int] = Stack()
tower_c: Stack[int] = Stack()
for i in range(1, disc_num + 1):
    tower_a.push(i)
print(tower_a)

[1, 2, 3]


In [142]:
def hanoi(begin: Stack[int], end: Stack[int], tmp: Stack[int], n:int)->None:
    if n == 1:
        end.push(begin.pop())
    else:
        hanoi(begin, tmp, end, n-1)
        hanoi(begin, end, tmp, 1)
        hanoi(tmp, end, begin, n-1)

In [143]:
%%time
hanoi(tower_a, tower_b, tower_c, disc_num)
print(tower_a)
print(tower_b)
print(tower_c)

[]
[1, 2, 3]
[]
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 275 µs


In [149]:
from enum import IntEnum
from typing import Tuple, List
Nucleotide: IntEnum = IntEnum("Nucleotide", ("A", "C", "G", "T"))
Codon = Tuple[Nucleotide, Nucleotide, Nucleotide]
Gene = List[Codon]

In [156]:
gene_str: str = "AAAGCCCGGGTCACGATACGGACTGATAATTTCAGGCATGCTAGTCAGTCGATGCTAGCAGTGCTATTCATG"
def string_to_gene(s: str)->Gene:
    gene: Gene = []
    for i in range(0, len(s), 3):
        if (i+2) >= len(s):
            return gene
        codon: Codon = (Nucleotide[s[i]], Nucleotide[s[i+1]], Nucleotide[s[i+2]])
        gene.append(codon)
    return gene

In [157]:
my_gene = string_to_gene(gene_str)

In [ ]:
def linear_contains(gene: Gene, key_codon: Codon)->bool:
    for codon in gene:
        if codon == key_codon:
            return True
    return False

def binary_contains(gene: Gene, key_codon: Codon)->bool:
    low: int = 0
    high: int = len(gene) - 1
    while low <= high:
        mid: int = (low + high) // 2
        if gene[mid] < key_codon:
            low = mid + 1
        elif gene[mid] > key_codon:
            high = mid - 1
        else:
            return True
    return False